- model cfg
    - https://transformerlensorg.github.io/TransformerLens/generated/code/transformer_lens.HookedTransformerConfig.html
- `logits, cache = mode.run_witch_cache(dataset)`
    - 运行模型时同时捕获中间计算结果（通常是某些层或模块的输出）以便后续分析和调试
    - `cache["pattern", layer, "attn"]`: 从指定的 layer 层获取注意力模块的注意力模式（权重）。
        - 形状为 [batch_size, num_heads, seq_length, seq_length] 的张量，包含注意力权重。
    - `cache["pre", layer, "mlp"]`: 从指定的 layer 层获取 MLP 模块的预激活输出。
        - 形状为 [batch_size, seq_length, d_model] 的张量，包含在激活函数应用之前的值。
    - `cache["post", layer, "mlp"]`: 从指定的 layer 层获取 MLP 模块的后激活输出。
        - 形状为 [batch_size, seq_length, d_model] 的张量，包含在激活函数应用之后的值。

### modules

- Embed()/Unembed()
    - Embed: W_E
    - Unembed: W_U
- MLP()
    - W_in
    - W_out
- Attention()
    - QKVO:
        - W(weights)
            - W_Q, W_K, W_V, W_O
        - b(biases)
            - b_Q, b_K, b_V, b_O